## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd #Fazer leitura do CSV e devolve um dataframe

# 1. Lê o CSV
df = pd.read_csv('SDW2025.csv', encoding='latin1', sep=';')

users = []

# 2. Transforma cada linha em um dicionário estruturado
for _, row in df.iterrows():
    user = {
        "id": row['UserID'],
        "name": row['Name'],
        "account": {
            "number": row['account_number'],
            "agency": row['account_agency']
        },
        "card": {
            "number": row['card_number'],
            "limit": row['card_limit']
        },
        "news": []
    }
    users.append(user)

# Agora a variável 'users' está idêntica ao que a API retornaria!
print(users[0])

{'id': 1, 'name': 'Julio', 'account': {'number': '00001-1', 'agency': 1}, 'card': {'number': '**** **** **** 1111', 'limit': 1000.0}, 'news': []}


In [ ]:
#Esse segundo passo seria obter os dados de cada ID usando a API da SDW
#Como a API não funciona mais, teremos que modificar como feito antes

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
#Integração com a API da OpenIA
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [ ]:
import openai

openai.api_key = openai_api_key #Configuramos a chave da API

#Fazendo o prompt de entrada para o sistema:
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para{user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )

  responseChatGPT = completion.choices[0].message.content.strip('\"') # Assim visualizamos as respostas
  return responseChatGPT

for user in users:
  news = generate_ai_news(user)
  print(news)
  users['news'].append({
      "Description": news
  })


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
import json

def save_users_to_json(users):
    with open('SDW2025_finalizado.json', 'w', encoding='utf-8') as file:
        json.dump(users, file, ensure_ascii=False,indent=2)
    return True

#Execução
success = save_users_to_json(users)
print(f"Processo ETL concluído! Dados salvos em JSON: {success}")
